### Packages

In [202]:
import os 
import pandas as pd 
import numpy as np 
import glob
import multiprocessing
import collections
from time import time
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup

# Graphes 
from graph_tool.all import *

# Scikit-Learn 
from sklearn.preprocessing import LabelEncoder


# Working Directory
os.chdir("/home/nbernard/Documents/data")

# POur les opcodes 

from pyevmasm import instruction_tables, disassemble_hex, disassemble_all, assemble_hex
import binascii


# Import de Bytecode de Smart Contracts

In [2]:
def aggregate_contract_csv():
    """
    Concatene les csv présents dans le répertoire courant, 
    Renvoie un pandas DF résultant de la concaténation
    
    """
    extension = 'csv'
    all_filenames = [i for i in glob.glob('contracts*.{}'.format(extension))]
    combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
    combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')
    return(combined_csv)

In [3]:
smc_data = pd.read_csv('contracts1.csv')

In [4]:
smc_data.head()

,address,bytecode,function_sighashes,is_erc20,is_erc721
0,0x38327f7f2c6163fc253e1fcdd4d21a833aabe328,0x606060405236156100f8576000357c01000000000000...,"0x173825d9,0x2f54bf6e,0x4123cb6b,0x52375093,0x...",False,False
1,0x623debf6ad97ba0fb39bdd7102a9fe9e5b7c4f51,0x6060604052361561001f5760e060020a6000350463f5...,0xf5537ede,False,False
2,0x0a72894c789b880e4bdeac91807c43cdfebc0c57,0x606060405236156100f8576000357c01000000000000...,"0x173825d9,0x2f54bf6e,0x4123cb6b,0x52375093,0x...",False,False
3,0x2ae0f168cf0a23b69e4c65a11837a28a6337d24b,0x606060405236156100f8576000357c01000000000000...,"0x173825d9,0x2f54bf6e,0x4123cb6b,0x52375093,0x...",False,False
4,0x345a6e3b030da16bcf121b315f0a9e7f52f89619,0x606060405236156100f8576000357c01000000000000...,"0x173825d9,0x2f54bf6e,0x4123cb6b,0x52375093,0x...",False,False


# Levenshtein Distance

In [5]:
def distance_leven(mot1, mot2):
    #print(os.getpid())
    dist = { (-1,-1): 0 }
    for i,c in enumerate(mot1) :
        dist[i,-1] = dist[i-1,-1] + 1
        dist[-1,i] = dist[-1,i-1] + 1
        for j,d in enumerate(mot2) :
            opt = [ ]
            if (i-1,j) in dist : 
                x = dist[i-1,j] + 1
                opt.append(x)
            if (i,j-1) in dist : 
                x = dist[i,j-1] + 1
                opt.append(x)
            if (i-1,j-1) in dist :
                x = dist[i-1,j-1] + (1 if c != d else 0)
                opt.append(x)
            dist[i,j] = min(opt)
    #print('done')
    return dist[len(mot1)-1,len(mot2)-1]/(len(mot1)+len(mot2))

# Ponzi Schemes Bytecodes and Opcodes

In [6]:
arrayponzi = {"object": "0x6060604052600060016000505560006003600050555b33600260006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908302179055505b6103d38061004f6000396000f36060604052361561005e576000357c010000000000000000000000000000000000000000000000000000000090048063365b98b21461027c57806361027f78146102c55780638da5cb5b146102e8578063bff1f9e1146103215761005e565b61027a5b670de0b6b3a764000034101561007757610002565b604060405190810160405280338152602001348152602001506000600050600060005080549050815481101561000257906000526020600020906002020160005b5060008201518160000160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908302179055506020820151816001016000505590505060016003600082828250540192505081905550600260009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff166000600a3404604051809050600060405180830381858888f19350505050505b60026000600050600160005054815481101561000257906000526020600020906002020160005b5060010160005054023073ffffffffffffffffffffffffffffffffffffffff16311115610277576000600050600160005054815481101561000257906000526020600020906002020160005b5060000160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16600060026000600050600160005054815481101561000257906000526020600020906002020160005b506001016000505402604051809050600060405180830381858888f193505050505060016001600082828250540192505081905550610166565b5b565b005b6102926004808035906020019091905050610344565b604051808373ffffffffffffffffffffffffffffffffffffffff1681526020018281526020019250505060405180910390f35b6102d2600480505061039b565b6040518082815260200191505060405180910390f35b6102f560048050506103a4565b604051808273ffffffffffffffffffffffffffffffffffffffff16815260200191505060405180910390f35b61032e60048050506103ca565b6040518082815260200191505060405180910390f35b600060005081815481101561000257906000526020600020906002020160005b915090508060000160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff16908060010160005054905082565b60016000505481565b600260009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1681565b6003600050548156","opcodes": "PUSH1 0x60 PUSH1 0x40 MSTORE PUSH1 0x0 PUSH1 0x1 PUSH1 0x0 POP SSTORE PUSH1 0x0 PUSH1 0x3 PUSH1 0x0 POP SSTORE JUMPDEST CALLER PUSH1 0x2 PUSH1 0x0 PUSH2 0x100 EXP DUP2 SLOAD DUP2 PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF MUL NOT AND SWAP1 DUP4 MUL OR SWAP1 SSTORE POP JUMPDEST PUSH2 0x3D3 DUP1 PUSH2 0x4F PUSH1 0x0 CODECOPY PUSH1 0x0 RETURN PUSH1 0x60 PUSH1 0x40 MSTORE CALLDATASIZE ISZERO PUSH2 0x5E JUMPI PUSH1 0x0 CALLDATALOAD PUSH29 0x100000000000000000000000000000000000000000000000000000000 SWAP1 DIV DUP1 PUSH4 0x365B98B2 EQ PUSH2 0x27C JUMPI DUP1 PUSH4 0x61027F78 EQ PUSH2 0x2C5 JUMPI DUP1 PUSH4 0x8DA5CB5B EQ PUSH2 0x2E8 JUMPI DUP1 PUSH4 0xBFF1F9E1 EQ PUSH2 0x321 JUMPI PUSH2 0x5E JUMP JUMPDEST PUSH2 0x27A JUMPDEST PUSH8 0xDE0B6B3A7640000 CALLVALUE LT ISZERO PUSH2 0x77 JUMPI PUSH2 0x2 JUMP JUMPDEST PUSH1 0x40 PUSH1 0x40 MLOAD SWAP1 DUP2 ADD PUSH1 0x40 MSTORE DUP1 CALLER DUP2 MSTORE PUSH1 0x20 ADD CALLVALUE DUP2 MSTORE PUSH1 0x20 ADD POP PUSH1 0x0 PUSH1 0x0 POP PUSH1 0x0 PUSH1 0x0 POP DUP1 SLOAD SWAP1 POP DUP2 SLOAD DUP2 LT ISZERO PUSH2 0x2 JUMPI SWAP1 PUSH1 0x0 MSTORE PUSH1 0x20 PUSH1 0x0 SHA3 SWAP1 PUSH1 0x2 MUL ADD PUSH1 0x0 JUMPDEST POP PUSH1 0x0 DUP3 ADD MLOAD DUP2 PUSH1 0x0 ADD PUSH1 0x0 PUSH2 0x100 EXP DUP2 SLOAD DUP2 PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF MUL NOT AND SWAP1 DUP4 MUL OR SWAP1 SSTORE POP PUSH1 0x20 DUP3 ADD MLOAD DUP2 PUSH1 0x1 ADD PUSH1 0x0 POP SSTORE SWAP1 POP POP PUSH1 0x1 PUSH1 0x3 PUSH1 0x0 DUP3 DUP3 DUP3 POP SLOAD ADD SWAP3 POP POP DUP2 SWAP1 SSTORE POP PUSH1 0x2 PUSH1 0x0 SWAP1 SLOAD SWAP1 PUSH2 0x100 EXP SWAP1 DIV PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND PUSH1 0x0 PUSH1 0xA CALLVALUE DIV PUSH1 0x40 MLOAD DUP1 SWAP1 POP PUSH1 0x0 PUSH1 0x40 MLOAD DUP1 DUP4 SUB DUP2 DUP6 DUP9 DUP9 CALL SWAP4 POP POP POP POP POP JUMPDEST PUSH1 0x2 PUSH1 0x0 PUSH1 0x0 POP PUSH1 0x1 PUSH1 0x0 POP SLOAD DUP2 SLOAD DUP2 LT ISZERO PUSH2 0x2 JUMPI SWAP1 PUSH1 0x0 MSTORE PUSH1 0x20 PUSH1 0x0 SHA3 SWAP1 PUSH1 0x2 MUL ADD PUSH1 0x0 JUMPDEST POP PUSH1 0x1 ADD PUSH1 0x0 POP SLOAD MUL ADDRESS PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND BALANCE GT ISZERO PUSH2 0x277 JUMPI PUSH1 0x0 PUSH1 0x0 POP PUSH1 0x1 PUSH1 0x0 POP SLOAD DUP2 SLOAD DUP2 LT ISZERO PUSH2 0x2 JUMPI SWAP1 PUSH1 0x0 MSTORE PUSH1 0x20 PUSH1 0x0 SHA3 SWAP1 PUSH1 0x2 MUL ADD PUSH1 0x0 JUMPDEST POP PUSH1 0x0 ADD PUSH1 0x0 SWAP1 SLOAD SWAP1 PUSH2 0x100 EXP SWAP1 DIV PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND PUSH1 0x0 PUSH1 0x2 PUSH1 0x0 PUSH1 0x0 POP PUSH1 0x1 PUSH1 0x0 POP SLOAD DUP2 SLOAD DUP2 LT ISZERO PUSH2 0x2 JUMPI SWAP1 PUSH1 0x0 MSTORE PUSH1 0x20 PUSH1 0x0 SHA3 SWAP1 PUSH1 0x2 MUL ADD PUSH1 0x0 JUMPDEST POP PUSH1 0x1 ADD PUSH1 0x0 POP SLOAD MUL PUSH1 0x40 MLOAD DUP1 SWAP1 POP PUSH1 0x0 PUSH1 0x40 MLOAD DUP1 DUP4 SUB DUP2 DUP6 DUP9 DUP9 CALL SWAP4 POP POP POP POP POP PUSH1 0x1 PUSH1 0x1 PUSH1 0x0 DUP3 DUP3 DUP3 POP SLOAD ADD SWAP3 POP POP DUP2 SWAP1 SSTORE POP PUSH2 0x166 JUMP JUMPDEST JUMPDEST JUMP JUMPDEST STOP JUMPDEST PUSH2 0x292 PUSH1 0x4 DUP1 DUP1 CALLDATALOAD SWAP1 PUSH1 0x20 ADD SWAP1 SWAP2 SWAP1 POP POP PUSH2 0x344 JUMP JUMPDEST PUSH1 0x40 MLOAD DUP1 DUP4 PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND DUP2 MSTORE PUSH1 0x20 ADD DUP3 DUP2 MSTORE PUSH1 0x20 ADD SWAP3 POP POP POP PUSH1 0x40 MLOAD DUP1 SWAP2 SUB SWAP1 RETURN JUMPDEST PUSH2 0x2D2 PUSH1 0x4 DUP1 POP POP PUSH2 0x39B JUMP JUMPDEST PUSH1 0x40 MLOAD DUP1 DUP3 DUP2 MSTORE PUSH1 0x20 ADD SWAP2 POP POP PUSH1 0x40 MLOAD DUP1 SWAP2 SUB SWAP1 RETURN JUMPDEST PUSH2 0x2F5 PUSH1 0x4 DUP1 POP POP PUSH2 0x3A4 JUMP JUMPDEST PUSH1 0x40 MLOAD DUP1 DUP3 PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND DUP2 MSTORE PUSH1 0x20 ADD SWAP2 POP POP PUSH1 0x40 MLOAD DUP1 SWAP2 SUB SWAP1 RETURN JUMPDEST PUSH2 0x32E PUSH1 0x4 DUP1 POP POP PUSH2 0x3CA JUMP JUMPDEST PUSH1 0x40 MLOAD DUP1 DUP3 DUP2 MSTORE PUSH1 0x20 ADD SWAP2 POP POP PUSH1 0x40 MLOAD DUP1 SWAP2 SUB SWAP1 RETURN JUMPDEST PUSH1 0x0 PUSH1 0x0 POP DUP2 DUP2 SLOAD DUP2 LT ISZERO PUSH2 0x2 JUMPI SWAP1 PUSH1 0x0 MSTORE PUSH1 0x20 PUSH1 0x0 SHA3 SWAP1 PUSH1 0x2 MUL ADD PUSH1 0x0 JUMPDEST SWAP2 POP SWAP1 POP DUP1 PUSH1 0x0 ADD PUSH1 0x0 SWAP1 SLOAD SWAP1 PUSH2 0x100 EXP SWAP1 DIV PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND SWAP1 DUP1 PUSH1 0x1 ADD PUSH1 0x0 POP SLOAD SWAP1 POP DUP3 JUMP JUMPDEST PUSH1 0x1 PUSH1 0x0 POP SLOAD DUP2 JUMP JUMPDEST PUSH1 0x2 PUSH1 0x0 SWAP1 SLOAD SWAP1 PUSH2 0x100 EXP SWAP1 DIV PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND DUP2 JUMP JUMPDEST PUSH1 0x3 PUSH1 0x0 POP SLOAD DUP2 JUMP "}

contract ArrayPonzi{
    struct User{
        address addr;
        uint amount;
    }
    User[] public users;
    uint public paying = 0;
    address public owner;
    uint public totalUsers=0;
    
    function ArrayPonzi(){
        owner = msg.sender;
    }
    function()  {
        if (msg.value < 1 ether) throw;
        
        users[users.length] = User({addr:msg.sender,amount:msg.value});
        totalUsers += 1 ;
        owner.send(msg.value/10);
        
        while (this.balance > users[paying].amount*2){
            users[paying].addr.send(users[paying].amount*2);
            paying += 1;
        }
        
    }
}

In [7]:
treeponzi = {
"object": "0x60606040525b60406040519081016040528033815260200133815260200150600060005060003373ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060005060008201518160000160006101000a81548173ffffffffffffffffffffffffffffffffffffffff0219169083021790555060208201518160010160006101000a81548173ffffffffffffffffffffffffffffffffffffffff0219169083021790555090505033600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908302179055505b61032c806100ec6000396000f360606040526000357c010000000000000000000000000000000000000000000000000000000090048063d014c01f1461003957610037565b005b61004f6004808035906020019091905050610051565b005b60006000670de0b6b3a76400003410806100d457506000600060005060003373ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060005060000160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1614155b8061014757506000600060005060008573ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060005060000160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16145b1561015157610002565b60406040519081016040528084815260200133815260200150600060005060003373ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060005060008201518160000160006101000a81548173ffffffffffffffffffffffffffffffffffffffff0219169083021790555060208201518160010160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908302179055509050508291503490505b600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168273ffffffffffffffffffffffffffffffffffffffff161415156102f35760028104905080508173ffffffffffffffffffffffffffffffffffffffff16600082604051809050600060405180830381858888f1935050505050600060005060008373ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060005060000160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1691508150610205565b8173ffffffffffffffffffffffffffffffffffffffff16600082604051809050600060405180830381858888f19350505050505b50505056",
"opcodes": "PUSH1 0x60 PUSH1 0x40 MSTORE JUMPDEST PUSH1 0x40 PUSH1 0x40 MLOAD SWAP1 DUP2 ADD PUSH1 0x40 MSTORE DUP1 CALLER DUP2 MSTORE PUSH1 0x20 ADD CALLER DUP2 MSTORE PUSH1 0x20 ADD POP PUSH1 0x0 PUSH1 0x0 POP PUSH1 0x0 CALLER PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND DUP2 MSTORE PUSH1 0x20 ADD SWAP1 DUP2 MSTORE PUSH1 0x20 ADD PUSH1 0x0 SHA3 PUSH1 0x0 POP PUSH1 0x0 DUP3 ADD MLOAD DUP2 PUSH1 0x0 ADD PUSH1 0x0 PUSH2 0x100 EXP DUP2 SLOAD DUP2 PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF MUL NOT AND SWAP1 DUP4 MUL OR SWAP1 SSTORE POP PUSH1 0x20 DUP3 ADD MLOAD DUP2 PUSH1 0x1 ADD PUSH1 0x0 PUSH2 0x100 EXP DUP2 SLOAD DUP2 PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF MUL NOT AND SWAP1 DUP4 MUL OR SWAP1 SSTORE POP SWAP1 POP POP CALLER PUSH1 0x1 PUSH1 0x0 PUSH2 0x100 EXP DUP2 SLOAD DUP2 PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF MUL NOT AND SWAP1 DUP4 MUL OR SWAP1 SSTORE POP JUMPDEST PUSH2 0x32C DUP1 PUSH2 0xEC PUSH1 0x0 CODECOPY PUSH1 0x0 RETURN PUSH1 0x60 PUSH1 0x40 MSTORE PUSH1 0x0 CALLDATALOAD PUSH29 0x100000000000000000000000000000000000000000000000000000000 SWAP1 DIV DUP1 PUSH4 0xD014C01F EQ PUSH2 0x39 JUMPI PUSH2 0x37 JUMP JUMPDEST STOP JUMPDEST PUSH2 0x4F PUSH1 0x4 DUP1 DUP1 CALLDATALOAD SWAP1 PUSH1 0x20 ADD SWAP1 SWAP2 SWAP1 POP POP PUSH2 0x51 JUMP JUMPDEST STOP JUMPDEST PUSH1 0x0 PUSH1 0x0 PUSH8 0xDE0B6B3A7640000 CALLVALUE LT DUP1 PUSH2 0xD4 JUMPI POP PUSH1 0x0 PUSH1 0x0 PUSH1 0x0 POP PUSH1 0x0 CALLER PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND DUP2 MSTORE PUSH1 0x20 ADD SWAP1 DUP2 MSTORE PUSH1 0x20 ADD PUSH1 0x0 SHA3 PUSH1 0x0 POP PUSH1 0x0 ADD PUSH1 0x0 SWAP1 SLOAD SWAP1 PUSH2 0x100 EXP SWAP1 DIV PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND EQ ISZERO JUMPDEST DUP1 PUSH2 0x147 JUMPI POP PUSH1 0x0 PUSH1 0x0 PUSH1 0x0 POP PUSH1 0x0 DUP6 PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND DUP2 MSTORE PUSH1 0x20 ADD SWAP1 DUP2 MSTORE PUSH1 0x20 ADD PUSH1 0x0 SHA3 PUSH1 0x0 POP PUSH1 0x0 ADD PUSH1 0x0 SWAP1 SLOAD SWAP1 PUSH2 0x100 EXP SWAP1 DIV PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND EQ JUMPDEST ISZERO PUSH2 0x151 JUMPI PUSH2 0x2 JUMP JUMPDEST PUSH1 0x40 PUSH1 0x40 MLOAD SWAP1 DUP2 ADD PUSH1 0x40 MSTORE DUP1 DUP5 DUP2 MSTORE PUSH1 0x20 ADD CALLER DUP2 MSTORE PUSH1 0x20 ADD POP PUSH1 0x0 PUSH1 0x0 POP PUSH1 0x0 CALLER PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND DUP2 MSTORE PUSH1 0x20 ADD SWAP1 DUP2 MSTORE PUSH1 0x20 ADD PUSH1 0x0 SHA3 PUSH1 0x0 POP PUSH1 0x0 DUP3 ADD MLOAD DUP2 PUSH1 0x0 ADD PUSH1 0x0 PUSH2 0x100 EXP DUP2 SLOAD DUP2 PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF MUL NOT AND SWAP1 DUP4 MUL OR SWAP1 SSTORE POP PUSH1 0x20 DUP3 ADD MLOAD DUP2 PUSH1 0x1 ADD PUSH1 0x0 PUSH2 0x100 EXP DUP2 SLOAD DUP2 PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF MUL NOT AND SWAP1 DUP4 MUL OR SWAP1 SSTORE POP SWAP1 POP POP DUP3 SWAP2 POP CALLVALUE SWAP1 POP JUMPDEST PUSH1 0x1 PUSH1 0x0 SWAP1 SLOAD SWAP1 PUSH2 0x100 EXP SWAP1 DIV PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND DUP3 PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND EQ ISZERO ISZERO PUSH2 0x2F3 JUMPI PUSH1 0x2 DUP2 DIV SWAP1 POP DUP1 POP DUP2 PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND PUSH1 0x0 DUP3 PUSH1 0x40 MLOAD DUP1 SWAP1 POP PUSH1 0x0 PUSH1 0x40 MLOAD DUP1 DUP4 SUB DUP2 DUP6 DUP9 DUP9 CALL SWAP4 POP POP POP POP POP PUSH1 0x0 PUSH1 0x0 POP PUSH1 0x0 DUP4 PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND DUP2 MSTORE PUSH1 0x20 ADD SWAP1 DUP2 MSTORE PUSH1 0x20 ADD PUSH1 0x0 SHA3 PUSH1 0x0 POP PUSH1 0x0 ADD PUSH1 0x0 SWAP1 SLOAD SWAP1 PUSH2 0x100 EXP SWAP1 DIV PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND SWAP2 POP DUP2 POP PUSH2 0x205 JUMP JUMPDEST DUP2 PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND PUSH1 0x0 DUP3 PUSH1 0x40 MLOAD DUP1 SWAP1 POP PUSH1 0x0 PUSH1 0x40 MLOAD DUP1 DUP4 SUB DUP2 DUP6 DUP9 DUP9 CALL SWAP4 POP POP POP POP POP JUMPDEST POP POP POP JUMP "
}

contract TreePonzi {
    struct User {
        address inviter;
        address itself;
    }
    mapping (address =>User) tree;
    address top;
    
    function TreePonzi(){
        tree[msg.sender] = User({itself:msg.sender,inviter:msg.sender});
        top = msg.sender;
    }
    
    function enter(address inviter) public {
        if ((msg.value < 1 ether) || (tree[msg.sender].inviter != 0x0) || (tree[inviter].inviter == 0x0)) throw;
        tree[msg.sender] = User({itself:msg.sender,inviter:inviter});
        address current = inviter;
        uint amount = msg.value;
        while(current != top){
            amount = amount/2;
            current.send(amount);
            current = tree[current].inviter;
        }
        current.send(amount);
}
    
}

In [8]:
handoverponzi = {
"object": "0x606060405267016345785d8a00006002600050555b33600060006101000a81548173ffffffffffffffffffffffffffffffffffffffff0219169083021790555033600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908302179055505b61025c806100796000396000f360606040523615610053576000357c0100000000000000000000000000000000000000000000000000000000900480630eb3f5a0146101095780634f8632ba14610121578063a035b1fe1461015a57610053565b6101075b60026000505434101561006957610002565b600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff166000600a6009340204604051809050600060405180830381858888f193505050505033600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908302179055506002600360026000505402046002600050819055505b565b005b61011f600480803590602001909190505061017d565b005b61012e600480505061022d565b604051808273ffffffffffffffffffffffffffffffffffffffff16815260200191505060405180910390f35b6101676004805050610253565b6040518082815260200191505060405180910390f35b600060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff16141561022957600060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16600082604051809050600060405180830381858888f19350505050505b5b50565b600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1681565b6002600050548156",
"opcodes": "PUSH1 0x60 PUSH1 0x40 MSTORE PUSH8 0x16345785D8A0000 PUSH1 0x2 PUSH1 0x0 POP SSTORE JUMPDEST CALLER PUSH1 0x0 PUSH1 0x0 PUSH2 0x100 EXP DUP2 SLOAD DUP2 PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF MUL NOT AND SWAP1 DUP4 MUL OR SWAP1 SSTORE POP CALLER PUSH1 0x1 PUSH1 0x0 PUSH2 0x100 EXP DUP2 SLOAD DUP2 PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF MUL NOT AND SWAP1 DUP4 MUL OR SWAP1 SSTORE POP JUMPDEST PUSH2 0x25C DUP1 PUSH2 0x79 PUSH1 0x0 CODECOPY PUSH1 0x0 RETURN PUSH1 0x60 PUSH1 0x40 MSTORE CALLDATASIZE ISZERO PUSH2 0x53 JUMPI PUSH1 0x0 CALLDATALOAD PUSH29 0x100000000000000000000000000000000000000000000000000000000 SWAP1 DIV DUP1 PUSH4 0xEB3F5A0 EQ PUSH2 0x109 JUMPI DUP1 PUSH4 0x4F8632BA EQ PUSH2 0x121 JUMPI DUP1 PUSH4 0xA035B1FE EQ PUSH2 0x15A JUMPI PUSH2 0x53 JUMP JUMPDEST PUSH2 0x107 JUMPDEST PUSH1 0x2 PUSH1 0x0 POP SLOAD CALLVALUE LT ISZERO PUSH2 0x69 JUMPI PUSH2 0x2 JUMP JUMPDEST PUSH1 0x1 PUSH1 0x0 SWAP1 SLOAD SWAP1 PUSH2 0x100 EXP SWAP1 DIV PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND PUSH1 0x0 PUSH1 0xA PUSH1 0x9 CALLVALUE MUL DIV PUSH1 0x40 MLOAD DUP1 SWAP1 POP PUSH1 0x0 PUSH1 0x40 MLOAD DUP1 DUP4 SUB DUP2 DUP6 DUP9 DUP9 CALL SWAP4 POP POP POP POP POP CALLER PUSH1 0x1 PUSH1 0x0 PUSH2 0x100 EXP DUP2 SLOAD DUP2 PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF MUL NOT AND SWAP1 DUP4 MUL OR SWAP1 SSTORE POP PUSH1 0x2 PUSH1 0x3 PUSH1 0x2 PUSH1 0x0 POP SLOAD MUL DIV PUSH1 0x2 PUSH1 0x0 POP DUP2 SWAP1 SSTORE POP JUMPDEST JUMP JUMPDEST STOP JUMPDEST PUSH2 0x11F PUSH1 0x4 DUP1 DUP1 CALLDATALOAD SWAP1 PUSH1 0x20 ADD SWAP1 SWAP2 SWAP1 POP POP PUSH2 0x17D JUMP JUMPDEST STOP JUMPDEST PUSH2 0x12E PUSH1 0x4 DUP1 POP POP PUSH2 0x22D JUMP JUMPDEST PUSH1 0x40 MLOAD DUP1 DUP3 PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND DUP2 MSTORE PUSH1 0x20 ADD SWAP2 POP POP PUSH1 0x40 MLOAD DUP1 SWAP2 SUB SWAP1 RETURN JUMPDEST PUSH2 0x167 PUSH1 0x4 DUP1 POP POP PUSH2 0x253 JUMP JUMPDEST PUSH1 0x40 MLOAD DUP1 DUP3 DUP2 MSTORE PUSH1 0x20 ADD SWAP2 POP POP PUSH1 0x40 MLOAD DUP1 SWAP2 SUB SWAP1 RETURN JUMPDEST PUSH1 0x0 PUSH1 0x0 SWAP1 SLOAD SWAP1 PUSH2 0x100 EXP SWAP1 DIV PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND CALLER PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND EQ ISZERO PUSH2 0x229 JUMPI PUSH1 0x0 PUSH1 0x0 SWAP1 SLOAD SWAP1 PUSH2 0x100 EXP SWAP1 DIV PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND PUSH1 0x0 DUP3 PUSH1 0x40 MLOAD DUP1 SWAP1 POP PUSH1 0x0 PUSH1 0x40 MLOAD DUP1 DUP4 SUB DUP2 DUP6 DUP9 DUP9 CALL SWAP4 POP POP POP POP POP JUMPDEST JUMPDEST POP JUMP JUMPDEST PUSH1 0x1 PUSH1 0x0 SWAP1 SLOAD SWAP1 PUSH2 0x100 EXP SWAP1 DIV PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND DUP2 JUMP JUMPDEST PUSH1 0x2 PUSH1 0x0 POP SLOAD DUP2 JUMP "
}

contract HandoverPonzi{
    address owner;
    address public user;
    uint public price = 100 finney;
    
    function HandoverPonzi(){
        owner = msg.sender;
        user = msg.sender;
    }
    
    function(){
        if (msg.value < price) throw;
        user.send(msg.value * 9/10);
        user = msg.sender ;
        price = price *3/2;
        
    }
    
    function sweepCommission(uint amount){
        if (msg.sender == owner) owner.send(amount);
        
    }
}

In [9]:
waterfallponzi = {
"object": "0x60606040526000600160005055600060026000505560006004600050555b33600360006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908302179055505b610406806100576000396000f36060604052361561005e576000357c010000000000000000000000000000000000000000000000000000000090048063365b98b2146102af5780638da5cb5b146102f85780639af1d35a14610331578063bff1f9e1146103545761005e565b6102ad5b670de0b6b3a764000034101561007757610002565b604060405190810160405280338152602001348152602001506000600050600260005054815481101561000257906000526020600020906002020160005b5060008201518160000160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908302179055506020820151816001016000505590505060016002600082828250540192505081905550600a3404600460005081905550600360009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff166000600460005054604051809050600060405180830381858888f193505050505060006001600050819055505b606460066000600050600160005054815481101561000257906000526020600020906002020160005b506001016000505402043073ffffffffffffffffffffffffffffffffffffffff1631101580156101df5750600260005054600160005054105b156102aa576000600050600160005054815481101561000257906000526020600020906002020160005b5060000160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff166000606460066000600050600160005054815481101561000257906000526020600020906002020160005b50600101600050540204604051809050600060405180830381858888f19350505050506001600160008282825054019250508190555061017d565b5b565b005b6102c56004808035906020019091905050610380565b604051808373ffffffffffffffffffffffffffffffffffffffff1681526020018281526020019250505060405180910390f35b61030560048050506103e0565b604051808273ffffffffffffffffffffffffffffffffffffffff16815260200191505060405180910390f35b61033e6004805050610377565b6040518082815260200191505060405180910390f35b61036160048050506103d7565b6040518082815260200191505060405180910390f35b60046000505481565b600060005081815481101561000257906000526020600020906002020160005b915090508060000160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff16908060010160005054905082565b60026000505481565b600360009054906101000a900473ffffffffffffffffffffffffffffffffffffffff168156",
"opcodes": "PUSH1 0x60 PUSH1 0x40 MSTORE PUSH1 0x0 PUSH1 0x1 PUSH1 0x0 POP SSTORE PUSH1 0x0 PUSH1 0x2 PUSH1 0x0 POP SSTORE PUSH1 0x0 PUSH1 0x4 PUSH1 0x0 POP SSTORE JUMPDEST CALLER PUSH1 0x3 PUSH1 0x0 PUSH2 0x100 EXP DUP2 SLOAD DUP2 PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF MUL NOT AND SWAP1 DUP4 MUL OR SWAP1 SSTORE POP JUMPDEST PUSH2 0x406 DUP1 PUSH2 0x57 PUSH1 0x0 CODECOPY PUSH1 0x0 RETURN PUSH1 0x60 PUSH1 0x40 MSTORE CALLDATASIZE ISZERO PUSH2 0x5E JUMPI PUSH1 0x0 CALLDATALOAD PUSH29 0x100000000000000000000000000000000000000000000000000000000 SWAP1 DIV DUP1 PUSH4 0x365B98B2 EQ PUSH2 0x2AF JUMPI DUP1 PUSH4 0x8DA5CB5B EQ PUSH2 0x2F8 JUMPI DUP1 PUSH4 0x9AF1D35A EQ PUSH2 0x331 JUMPI DUP1 PUSH4 0xBFF1F9E1 EQ PUSH2 0x354 JUMPI PUSH2 0x5E JUMP JUMPDEST PUSH2 0x2AD JUMPDEST PUSH8 0xDE0B6B3A7640000 CALLVALUE LT ISZERO PUSH2 0x77 JUMPI PUSH2 0x2 JUMP JUMPDEST PUSH1 0x40 PUSH1 0x40 MLOAD SWAP1 DUP2 ADD PUSH1 0x40 MSTORE DUP1 CALLER DUP2 MSTORE PUSH1 0x20 ADD CALLVALUE DUP2 MSTORE PUSH1 0x20 ADD POP PUSH1 0x0 PUSH1 0x0 POP PUSH1 0x2 PUSH1 0x0 POP SLOAD DUP2 SLOAD DUP2 LT ISZERO PUSH2 0x2 JUMPI SWAP1 PUSH1 0x0 MSTORE PUSH1 0x20 PUSH1 0x0 SHA3 SWAP1 PUSH1 0x2 MUL ADD PUSH1 0x0 JUMPDEST POP PUSH1 0x0 DUP3 ADD MLOAD DUP2 PUSH1 0x0 ADD PUSH1 0x0 PUSH2 0x100 EXP DUP2 SLOAD DUP2 PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF MUL NOT AND SWAP1 DUP4 MUL OR SWAP1 SSTORE POP PUSH1 0x20 DUP3 ADD MLOAD DUP2 PUSH1 0x1 ADD PUSH1 0x0 POP SSTORE SWAP1 POP POP PUSH1 0x1 PUSH1 0x2 PUSH1 0x0 DUP3 DUP3 DUP3 POP SLOAD ADD SWAP3 POP POP DUP2 SWAP1 SSTORE POP PUSH1 0xA CALLVALUE DIV PUSH1 0x4 PUSH1 0x0 POP DUP2 SWAP1 SSTORE POP PUSH1 0x3 PUSH1 0x0 SWAP1 SLOAD SWAP1 PUSH2 0x100 EXP SWAP1 DIV PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND PUSH1 0x0 PUSH1 0x4 PUSH1 0x0 POP SLOAD PUSH1 0x40 MLOAD DUP1 SWAP1 POP PUSH1 0x0 PUSH1 0x40 MLOAD DUP1 DUP4 SUB DUP2 DUP6 DUP9 DUP9 CALL SWAP4 POP POP POP POP POP PUSH1 0x0 PUSH1 0x1 PUSH1 0x0 POP DUP2 SWAP1 SSTORE POP JUMPDEST PUSH1 0x64 PUSH1 0x6 PUSH1 0x0 PUSH1 0x0 POP PUSH1 0x1 PUSH1 0x0 POP SLOAD DUP2 SLOAD DUP2 LT ISZERO PUSH2 0x2 JUMPI SWAP1 PUSH1 0x0 MSTORE PUSH1 0x20 PUSH1 0x0 SHA3 SWAP1 PUSH1 0x2 MUL ADD PUSH1 0x0 JUMPDEST POP PUSH1 0x1 ADD PUSH1 0x0 POP SLOAD MUL DIV ADDRESS PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND BALANCE LT ISZERO DUP1 ISZERO PUSH2 0x1DF JUMPI POP PUSH1 0x2 PUSH1 0x0 POP SLOAD PUSH1 0x1 PUSH1 0x0 POP SLOAD LT JUMPDEST ISZERO PUSH2 0x2AA JUMPI PUSH1 0x0 PUSH1 0x0 POP PUSH1 0x1 PUSH1 0x0 POP SLOAD DUP2 SLOAD DUP2 LT ISZERO PUSH2 0x2 JUMPI SWAP1 PUSH1 0x0 MSTORE PUSH1 0x20 PUSH1 0x0 SHA3 SWAP1 PUSH1 0x2 MUL ADD PUSH1 0x0 JUMPDEST POP PUSH1 0x0 ADD PUSH1 0x0 SWAP1 SLOAD SWAP1 PUSH2 0x100 EXP SWAP1 DIV PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND PUSH1 0x0 PUSH1 0x64 PUSH1 0x6 PUSH1 0x0 PUSH1 0x0 POP PUSH1 0x1 PUSH1 0x0 POP SLOAD DUP2 SLOAD DUP2 LT ISZERO PUSH2 0x2 JUMPI SWAP1 PUSH1 0x0 MSTORE PUSH1 0x20 PUSH1 0x0 SHA3 SWAP1 PUSH1 0x2 MUL ADD PUSH1 0x0 JUMPDEST POP PUSH1 0x1 ADD PUSH1 0x0 POP SLOAD MUL DIV PUSH1 0x40 MLOAD DUP1 SWAP1 POP PUSH1 0x0 PUSH1 0x40 MLOAD DUP1 DUP4 SUB DUP2 DUP6 DUP9 DUP9 CALL SWAP4 POP POP POP POP POP PUSH1 0x1 PUSH1 0x1 PUSH1 0x0 DUP3 DUP3 DUP3 POP SLOAD ADD SWAP3 POP POP DUP2 SWAP1 SSTORE POP PUSH2 0x17D JUMP JUMPDEST JUMPDEST JUMP JUMPDEST STOP JUMPDEST PUSH2 0x2C5 PUSH1 0x4 DUP1 DUP1 CALLDATALOAD SWAP1 PUSH1 0x20 ADD SWAP1 SWAP2 SWAP1 POP POP PUSH2 0x380 JUMP JUMPDEST PUSH1 0x40 MLOAD DUP1 DUP4 PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND DUP2 MSTORE PUSH1 0x20 ADD DUP3 DUP2 MSTORE PUSH1 0x20 ADD SWAP3 POP POP POP PUSH1 0x40 MLOAD DUP1 SWAP2 SUB SWAP1 RETURN JUMPDEST PUSH2 0x305 PUSH1 0x4 DUP1 POP POP PUSH2 0x3E0 JUMP JUMPDEST PUSH1 0x40 MLOAD DUP1 DUP3 PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND DUP2 MSTORE PUSH1 0x20 ADD SWAP2 POP POP PUSH1 0x40 MLOAD DUP1 SWAP2 SUB SWAP1 RETURN JUMPDEST PUSH2 0x33E PUSH1 0x4 DUP1 POP POP PUSH2 0x377 JUMP JUMPDEST PUSH1 0x40 MLOAD DUP1 DUP3 DUP2 MSTORE PUSH1 0x20 ADD SWAP2 POP POP PUSH1 0x40 MLOAD DUP1 SWAP2 SUB SWAP1 RETURN JUMPDEST PUSH2 0x361 PUSH1 0x4 DUP1 POP POP PUSH2 0x3D7 JUMP JUMPDEST PUSH1 0x40 MLOAD DUP1 DUP3 DUP2 MSTORE PUSH1 0x20 ADD SWAP2 POP POP PUSH1 0x40 MLOAD DUP1 SWAP2 SUB SWAP1 RETURN JUMPDEST PUSH1 0x4 PUSH1 0x0 POP SLOAD DUP2 JUMP JUMPDEST PUSH1 0x0 PUSH1 0x0 POP DUP2 DUP2 SLOAD DUP2 LT ISZERO PUSH2 0x2 JUMPI SWAP1 PUSH1 0x0 MSTORE PUSH1 0x20 PUSH1 0x0 SHA3 SWAP1 PUSH1 0x2 MUL ADD PUSH1 0x0 JUMPDEST SWAP2 POP SWAP1 POP DUP1 PUSH1 0x0 ADD PUSH1 0x0 SWAP1 SLOAD SWAP1 PUSH2 0x100 EXP SWAP1 DIV PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND SWAP1 DUP1 PUSH1 0x1 ADD PUSH1 0x0 POP SLOAD SWAP1 POP DUP3 JUMP JUMPDEST PUSH1 0x2 PUSH1 0x0 POP SLOAD DUP2 JUMP JUMPDEST PUSH1 0x3 PUSH1 0x0 SWAP1 SLOAD SWAP1 PUSH2 0x100 EXP SWAP1 DIV PUSH20 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF AND DUP2 JUMP "
}

contract WaterfallPonzi{
    struct User {
        address addr;
        uint amount;
    }
    
    User[] public users;
    uint pos = 0;
    uint public totalUsers=0;
    address public owner;
    uint public fees = 0;
    
    function WaterfallPonzi(){
        owner = msg.sender;
    }
    
    function(){
        if (msg.value < 1 ether) throw;
        
        users[totalUsers] = User({addr:msg.sender,amount:msg.value});
        
        totalUsers += 1;
        fees = msg.value/10;
        owner.send(fees);
        
        pos = 0;
        while (this.balance >= users[pos].amount * 6/100 && pos < totalUsers){
            users[pos].addr.send(users[pos].amount * 6/100);
            pos += 1;
        }
    }
}

# Création de la distance 

In [10]:
# On garde en mémoire l'add upcode

def add_dist_parr2(df):
    
    """
    Version parallélisée
    A partir d'un DF pandas, ajoute la distance de levenshtein des bytecodes des contrats aux bytecodes de référence
    
    """
    # Liste des bytecode de référence
    
    arr = arrayponzi['object']
    tree = treeponzi['object']
    handover = handoverponzi['object']
    water = waterfallponzi['object']
    
    
    ponzi_list = [arr,tree,handover,water]
    time_elapsed1 = 0
    result_list = []
    
    for ponz_type in ponzi_list:
        
        pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
        start = time()
        results = []
        
        for m1 in tqdm(df['bytecode']):
            results.append(pool.apply(distance_leven,args=(m1,ponz_type)))
        
        
        end = time()
        time_elapsed1 += (end-start)
        
        result_list.append(results)
        
        pool.close()
    
    df['byte dist arr'] = result_list[0]
    df['byte dist tree'] = result_list[1]
    df['byte dist handover'] = result_list[2]
    df['byte dist water'] = result_list[3]
    
    # Comparaison des OPCODES 
    
    arr = arrayponzi['opcodes']
    tree = treeponzi['opcodes']
    handover = handoverponzi['opcodes']
    water = waterfallponzi['opcodes']
    
    
    ponzi_list = [arr,tree,handover,water]
    time_elapsed2 = 0
    result_list = []
    
    for ponz_type in ponzi_list:
        
        pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
        start = time()
        results = []
        
        for m1 in tqdm(df['opcode']):
            results.append(pool.apply(distance_leven,args=(m1,ponz_type)))
        
        
        end = time()
        time_elapsed2 += (end-start)
        
        result_list.append(results)
        
        pool.close()
    
    df['op dist arr'] = result_list[0]
    df['op dist tree'] = result_list[1]
    df['op dist handover'] = result_list[2]
    df['op dist water'] = result_list[3]    
    
    return(df,time_elapsed1+time_elapsed2)

In [11]:
def add_dist_parr(df):
    
    """
    Version parallélisée
    A partir d'un DF pandas, ajoute la distance de levenshtein des bytecodes des contrats aux bytecodes de référence
    
    """
    # Liste des bytecode de référence
    
    arr = arrayponzi['object']
    tree = treeponzi['object']
    handover = handoverponzi['object']
    water = waterfallponzi['object']
    
    
    ponzi_list = [arr,tree,handover,water]
    time_elapsed1 = 0
    result_list = []
    
    for ponz_type in ponzi_list:
        
        pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
        start = time()
        results = []
        
        for m1 in tqdm(df['bytecode']):
            results.append(pool.apply(distance_leven,args=(m1,ponz_type)))
        
        
        end = time()
        time_elapsed1 += (end-start)
        
        result_list.append(results)
        
        pool.close()
    
    df['byte dist arr'] = result_list[0]
    df['byte dist tree'] = result_list[1]
    df['byte dist handover'] = result_list[2]
    df['byte dist water'] = result_list[3]
    
    print(time_elapsed1+time_elapsed2)
    
    return(df)

# Enrichissement des données sur les Smart Contract

## Rajout des OPCODES

In [12]:
#x = df.iloc[3].bytecode

In [13]:
#x = "0x60606040525b60406040519081016040528033815260200133815260200150600060005060003373ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060005060008201518160000160006101000a81548173ffffffffffffffffffffffffffffffffffffffff0219169083021790555060208201518160010160006101000a81548173ffffffffffffffffffffffffffffffffffffffff0219169083021790555090505033600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908302179055505b61032c806100ec6000396000f360606040526000357c010000000000000000000000000000000000000000000000000000000090048063d014c01f1461003957610037565b005b61004f6004808035906020019091905050610051565b005b60006000670de0b6b3a76400003410806100d457506000600060005060003373ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060005060000160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1614155b8061014757506000600060005060008573ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060005060000160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16145b1561015157610002565b60406040519081016040528084815260200133815260200150600060005060003373ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060005060008201518160000160006101000a81548173ffffffffffffffffffffffffffffffffffffffff0219169083021790555060208201518160010160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908302179055509050508291503490505b600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168273ffffffffffffffffffffffffffffffffffffffff161415156102f35760028104905080508173ffffffffffffffffffffffffffffffffffffffff16600082604051809050600060405180830381858888f1935050505050600060005060008373ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060005060000160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1691508150610205565b8173ffffffffffffffffffffffffffffffffffffffff16600082604051809050600060405180830381858888f19350505050505b50505056"

In [14]:
#instrs = list(disassemble_all(binascii.unhexlify(x[2:])))

In [15]:
#a = assemble_hex(instrs)

In [16]:
#print(disassemble_hex(a))

In [17]:
#str(disassemble_hex(a))

In [18]:
#str(disassemble_hex(a)).replace("\n"," ")

In [19]:
def add_opcodes(df):
    """
    Ajoute à un DF Pandas une colonne avec les opcodes déduits du bytecode associé à chaque contrat
    
    """
    opcodes = []
    start = time()
    for bytecode in df['bytecode']:
        if bytecode != '0x':
            
            #instrs = list(disassemble_all(binascii.unhexlify(bytecode[2:])))
            #a = assemble_hex(instrs)
        
            res = str(disassemble_hex(bytecode[2:])).replace("\n"," ")
            opcodes.append(res)
        else:
            opcodes.append(" ")
        
    df['opcode'] = opcodes
    end = time()
    
    print("Time spent on adding opcodes:  " + str(end-start))
    
    return(df)

## Rajout des transactions

In [20]:
def get_transactions(df):
    """
    EXTRACTION DU NOMBRE DE  TXNS
    NB: IL Y A UN ECART ENTRE LE NB OBTENU SUR ETHERSCAN ET INFURA, ON PREND LA VALEUR ETHERSCAN LORS DE LA FUSION
    """
    API_ENDPOINT = "https://mainnet.infura.io/v3/c6a6f0e057ec465b90b5b85c5cdb5990"
    
    data_list = []
    for j in df.address:
        data_list.append({"jsonrpc":"2.0","method":"eth_getTransactionCount","params": [j,"latest"],"id":1})
    
    res_list = []

    for j in data_list:
        data = j
        headers = {"Content-Type": "application/json"}
    
        # sending post request and saving response as response object 
        r = requests.post(url = API_ENDPOINT, json = data, headers=headers) 
  
        # extracting response text  
        pastebin_url = r.text 

        dict_res = eval(pastebin_url)
        res = dict_res['result']
        res = int(res,16)
        res_list.append(res)
        
    
    df['txn'] = res_list
    
    return(df)

In [21]:
#smc_data = get_transactions(smc_data)

In [22]:
#smc_data['txn']

## Rajout des balances

In [23]:
def get_balance(df):
    """
    On complète le tableau avec les balances et les transactions 
    https://api.etherscan.io/api?module=account&action=balancemulti&address={}&tag=latest&apikey=B16PQFQJ9UPYNBP8R6YXF3631EVRTBHUFZ
    EXTRACTION DES BALANCES 
    """
    
    urls = []
    n = 1
    url = str()
    n_iters = np.ceil(df.shape[0]/20) # Le nombre de requetes nécessaires, on ne peut en faire que par batch de 20
    compteur = 0
    
    for add in df.address:
        if n%20 != 0:
            url = url + add +',' 
            n += 1
            
            
            
            
        else:
            url = url + add 
            urls.append(url)
            url = str()
            n = 1
            compteur += 1 
            
    if compteur != n_iters:
        url = str()
        for add in df.address[(compteur*20):]:
            url = url + add +',' 
            
        url = url[:-1]    
        urls.append(url)
    
    
    balance = []
    for url in urls:
        
        # adresse de l'api
        current_url = "https://api.etherscan.io/api?module=account&action=balancemulti&address={}&tag=latest&apikey=B16PQFQJ9UPYNBP8R6YXF3631EVRTBHUFZ".format(url)
        r = requests.get(url = current_url) 
  
        # extraction du json
        data = r.json() 
    
        res = data['result']
        df_res = pd.DataFrame(data=res)
        df_res['balance'] = df_res['balance'].astype(float)*10**(-18)
        curr_bal = list(df_res['balance'].values)
        balance += curr_bal
        
    df['balance'] = balance
    
    return(df)

In [24]:
#smc_data = pd.read_csv('contracts1.csv')

In [25]:
#smc_data = get_balance(smc_data)

In [26]:
#smc_data

In [27]:
#smc_data[smc_data['balance'] != 0]

## Rajout des Transactions Paralleles + Distance a l'exchange

### Import des données préalable

In [28]:
def get_earliest_block(df):
    block_number = np.min(test_data['block_number'] )
    return(block_number)

In [29]:
def load_data(block_number):
    
    os.chdir("/home/nbernard/output/transactions")
    csv_list = []
    data = pd.DataFrame()
    
    for start in sorted(os.listdir()):
        folder_list = os.listdir("/home/nbernard/output/contracts/{}".format(start))
        folder = sorted(os.listdir("/home/nbernard/output/contracts/{}/{}".format(start,folder_list[0])))
        csv_list.append(folder)
        
    csv_list = np.array(csv_list).flatten()
    l = [k for k in range(0,5010000,100000)]
    
    for k in range(len(l)-1):
        if block_number >= l[k] and block_number < l[k+1]:
            number = k
        else: 
            if block_number >= l[k] and block_number == l[k+1]:
                number = k+1
    
    # On prend a partir de number (inclus)
    
    load_list = csv_list[number:]
    
    for k in range(len(load_list)):
        load_list[k] = load_list[k].replace('contracts','transactions')
    
    start_list = []
    end_list = []
    
    for tr in load_list:
        n = str()
        tr = tr.replace("_",' ')
        
        for s in range(len(tr)):
            if tr[s].isdigit():
                n += (tr[s])
        start_list.append(n[0:8])
        end_list.append(n[8:])
    
    print(start_list)
    print(end_list)
    for start,end in zip(start_list,end_list):
        
        os.chdir("/home/nbernard/output/transactions/start_block={}/end_block={}".format(start,end))
        df = pd.read_csv("transactions_{}_{}.csv".format(start,end))
        data = data.append(df)
        
    return(data)

In [30]:
def format_as_tgraph(data):
    """
    Formatage d'un pandas DF en un pandas DF adapté
    à un import en graphe de transactions: 
    
    Renvoie un pandas DF dans un format adapté a un import graphique
    
    """
    
    # On transforme les addresses en str pour éviter les ennuis
    data[['from_address', 'to_address']] = data[['from_address', 'to_address']].astype(str)

    
    # On concatene les addresses pour éviter les doublons 
    
    full_addresses = np.append(data['from_address'].values,data['to_address'].values)
    
    
    # Encodage des addresses sur la liste ENTIERE des addresses 
    Encoder = LabelEncoder()
    Encoder.fit(full_addresses) #Apprend sur les addresses 

    new_from = Encoder.transform(data['from_address']) # On transforme
    new_to = Encoder.transform(data['to_address'])     # On transforme
    
    
    # Remplacement des colonnes par les colonnes ainsi encodées
    data['from_address'] = new_from
    data['to_address'] = new_to
    
    # On transforme la colonne value en float
    data[['value']] = data[['value']].astype(float)
    
    
    return(data,Encoder)

In [31]:
def create_graph_obj(data):
    """
    Prend un pandas DF formaté correctement en transaction graph
    Crée et renvoie un objet graph-tool sur lequel on peut travailler
    
    """
    
    # Création du graphe vide
    g = Graph(directed=True) 

    # Ajout des arêtes suivant le np.array
    vmap = g.add_edge_list(data[['from_address','to_address']].values)

    # Ajout des edge weights 
    ew = g.new_edge_property("double")
    ew.a = data['value'].values
    g.ep['edge_weight'] = ew
    
    return(g)



### Calcul du nombre de transactions parallèles 

In [64]:
def parallel_tr(graph):
    """
    Prend en arguments le graphe de transactions à étudier
    Renvoie un tableau avec l'ensemble des transaction paralleles selon graph_tool
    Les transactions paralleles sont numerotees de 1 a (nb_transaction_par - 1) par chaque arete parallele reperee 
    
    """
    g = graph
    
    # On récupère les arêtes parallèles et on stocke dans un numpy array
    
    para = label_parallel_edges(g)
    nb_para = np.array(para.a)
    
    # On prépare le tableau   
    
    df_edges = pd.DataFrame(g.get_edges(),columns=['from','to','index']) # tableau complet des arêtes
    d_e = pd.DataFrame(np.arange(len(nb_para)),columns=['e_index']) # tableau des arêtes outliers  
    
    d_e.set_index('e_index',inplace=True) # changement des indexs pour un inner join
    df_edges.set_index('index',inplace=True) # changement des indexs pour un inner join
    
    df_edges_out = d_e.merge(df_edges,left_index=True,right_index=True) # jointure interieure
        
    # On réencode les adresses
    df_edges_out['from_address'] = Encoder.inverse_transform(df_edges_out['from'])
    df_edges_out['to_address'] = Encoder.inverse_transform(df_edges_out['to'])
    df_edges_out['number_parallel_transactions'] = nb_para
    
    return(df_edges_out)

In [ ]:
def add_parr(par_df2)
    start = time()

    k_used = set()
    for k in tqdm(par_df2['from']):
        if k not in k_used:
            j_used = set()
            for j in tqdm(par_df2[par_df2['from'] == k]['to']):
                if j not in j_used:
                    value = np.max(par_df2[(par_df2['from'] == k) & (par_df2['to'] == j) ]['number_parallel_transactions'])
                    par_df2.loc[(par_df2['from'] == k) & (par_df2['to'] == j),'number_parallel_transactions'] = value+1 
                    j_used.add(j)
                
            k_used.add(k)
        
    end = time()
    print(end-start)
    
    return(par_df2)

### Calcul de la distance aux exchanges

In [200]:
def recup_nb_pages(start_page="https://etherscan.io/accounts/label/phish-hack?sort=balance&order=desc&ps=100"):
    
    
    source = requests.get(start_page).text
    soup = BeautifulSoup(source,'lxml') 
    nb_page = soup.find_all('strong',class_="font-weight-medium") # on récupère la balise 
    nb = int(remove_html_markup(nb_page[1])) 
    
    return(nb)
    
    
    

In [204]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
            if c == '<' and not quote:
                tag = True
            elif c == '>' and not quote:
                tag = False
            elif (c == '"' or c == "'") and tag:
                quote = not quote
            elif not tag:
                out = out + c

    return out

In [196]:
def parse_exchange(start_page="https://etherscan.io/accounts/label/exchange/1?ps=100"):
    
    source = requests.get(start_page).text
    soup = BeautifulSoup(source,'lxml') 
    extract_data = soup.find('div',class_="table-responsive mb-2 mb-md-0") # on récupère l'ensemble du tableau
    tableau = extract_data.find('tbody') # on récupère le corps du tableau
    lignes = tableau.find_all('tr')
    addresses = []
    name = []
    balance = []
    txn = []

    for i in lignes:
        add = i.find('td').find('a') # Address
        nam = i.find_all('td')[1] # Nom
        bal = i.find_all('td')[2] # balance
        tx = i.find_all('td')[3] # TxN count
    
        add = remove_html_markup(add)
        nam = remove_html_markup(nam)
        bal = remove_html_markup(bal.text)
        tx = remove_html_markup(tx)
    
        addresses.append(add)
        name.append(nam)
        balance.append(bal)
        txn.append(tx)
    
    return(addresses,name,balance,txn)

In [197]:
def parse_all_exchanges():
    
    addresses = []
    name = []
    balance = []
    txn = []
    
    # Nombre de pages
    nb_pages = recup_nb_pages("https://etherscan.io/accounts/label/exchange/1?ps=100")
    
    nb_pagelist = [i+1 for i in range(nb_pages)]    
    
    for i in nb_pagelist:
        page = "https://etherscan.io/accounts/label/exchange/{}?ps=100".format(i)
        add,nam,bal,tx = parse_exchange(page) 
        addresses += add
        name += nam
        balance += bal
        txn += tx
    

    # Formatage de balance et de txn
    for i in range(len(balance)):
        if "Ether" in balance[i]:
            balance[i] = balance[i].strip("Ether")
            balance[i] = float(balance[i].replace(',', ''))
            
        else:
            balance[i] = balance[i].strip("wei")
            balance[i] = float(balance[i].replace(',', ''))
            balance[i] = balance[i]*10**(-18)
        
        if "," in txn[i]:
            txn[i] = int(txn[i].replace(',', ''))
        else:
            txn[i] = int(txn[i])
    
    
    # tableau pandas 
    data = {"addresses":addresses,"name":name,"balance":balance,"txn":txn}
    df = pd.DataFrame(data=data)
    
    return(df)    

## TF-IDF sur les OPCODES

In [32]:
# TF-ID des opcodes

def add_tf_id(df):
    """
    
    Prend un pandas df contenant les opcodes et renvoie un dataframe augmente contenant les tokens tf-id associés
    Penser a ajouter verif que la colonne opcode existe.
    
    """
    
    tf = TfidfVectorizer()
    tfid_matrix = tf.fit_transform(raw_documents=df['opcode'])
    
    aug_df = pd.SparseDataFrame(tfid_matrix,
                         columns=tf.get_feature_names(), 
                         default_fill_value=0)
    
    
    for columns in df.columns:
        aug_df[str(k)] = df[str(k)]
    
    
    return(aug_df)

# TEST

In [33]:
data = load_data(4100000)

['04100000']
['04199999']


In [34]:
data

,hash,nonce,block_hash,block_number,transaction_index,from_address,to_address,value,gas,gas_price,input
0,0x0bfb813c50d7e37b7c37df5ee33f90a659e0687071e0...,81417,0x9d15648d01925f689f66d689299b33ac6a3a86286782...,4100200,0,0x7ed1e469fcb3ee19c0366d829e291451be638e59,0xf72d552ad28e2ea7db6ca5794d359aec190d9151,162000000000000000,21000,60000000000,0x
1,0x165a95bb79d39597ca95eaae45d4baae1828deb1d82c...,630827,0x9d15648d01925f689f66d689299b33ac6a3a86286782...,4100200,1,0x32be343b94f860124dc4fee278fdcbd38c102d88,0x1e49a5551a183d16451982729ba03e5887364e58,492419350000000000,100000,51000000000,0x
2,0x18e5ac13b126f43c25c5c443f5a967336a3e58b077b0...,2,0x9d15648d01925f689f66d689299b33ac6a3a86286782...,4100200,2,0x32ec262a2022ba5adca1e3ad1b1fe85be63fdcdd,0xe94b04a0fed112f3664e45adb2b8915693dd5ff3,498242030000000000,135546,51000000000,0x0f2c9329000000000000000000000000fbb1b73c4f0b...
3,0x79c9060ad01926cab66835a615cfa0326f4bcaa505f8...,0,0x9d15648d01925f689f66d689299b33ac6a3a86286782...,4100200,3,0xf10a0907bc4d0953915c27641c8583aea8d78537,0xe94b04a0fed112f3664e45adb2b8915693dd5ff3,677899000000000000,135546,51000000000,0x0f2c9329000000000000000000000000fbb1b73c4f0b...
4,0x828ed0499dfa8793d7d97a0c113667ed86fef258115e...,2,0x9d15648d01925f689f66d689299b33ac6a3a86286782...,4100200,4,0x4614ecb1e4afd8fae805eb3e5548579e4d039464,0xe94b04a0fed112f3664e45adb2b8915693dd5ff3,1815742030000000000,135546,51000000000,0x0f2c9329000000000000000000000000fbb1b73c4f0b...
5,0x86da238e388b339fe9b8c057a32b683746f4fe2cbcdd...,365,0x9d15648d01925f689f66d689299b33ac6a3a86286782...,4100200,5,0x5837cb5d40a0ad9a92c61f670a30de7a05ac93a7,0xe94b04a0fed112f3664e45adb2b8915693dd5ff3,998267970000000000,135546,51000000000,0x0f2c9329000000000000000000000000fbb1b73c4f0b...
6,0x877428e16160c4276acdf4d5a059cd9aa1f486424f3a...,0,0x9d15648d01925f689f66d689299b33ac6a3a86286782...,4100200,6,0xa1e8d3d616516a7a9b64e75ffae7e3ab36f943ef,0xe94b04a0fed112f3664e45adb2b8915693dd5ff3,15349755890000000000,135546,51000000000,0x0f2c9329000000000000000000000000fbb1b73c4f0b...
7,0xa82713185a8984dacd5e0c6162e5fb1ebcf0527bc609...,5,0x9d15648d01925f689f66d689299b33ac6a3a86286782...,4100200,7,0x82ebc9ba24b9ea75a334367b49c54db6f96fa2ef,0xe94b04a0fed112f3664e45adb2b8915693dd5ff3,97308350000000000,135546,51000000000,0x0f2c9329000000000000000000000000fbb1b73c4f0b...
8,0xb2b65517065eb6b14a7ac7edb20f4635b8fb417873fc...,7,0x9d15648d01925f689f66d689299b33ac6a3a86286782...,4100200,8,0xba7c43a82738ce71164233db9619c10683fff6f4,0xe94b04a0fed112f3664e45adb2b8915693dd5ff3,97278990000000000,135546,51000000000,0x0f2c9329000000000000000000000000fbb1b73c4f0b...
9,0x0a81399916cd75ee323d09f9f3d222746dd58362cf55...,630828,0x9d15648d01925f689f66d689299b33ac6a3a86286782...,4100200,9,0x32be343b94f860124dc4fee278fdcbd38c102d88,0xc96d400ecc44dfec99742e91d99f54d2cbc47651,128133340000000000,100000,51000000000,0x


In [35]:
data,Encoder = format_as_tgraph(data)

In [36]:
g = create_graph_obj(data)

In [45]:
def parallel_tr(graph):
    """
    Prend en arguments le graphe de transactions à étudier, et lambda permettant d'ajuster le seuil 
    Renvoie les outliers au sens des transactions parallèles 
    sous forme d'un DF pandas 
    
    """
    g = graph
    
    # On récupère les arêtes parallèles et on stocke dans un numpy array
    
    para = label_parallel_edges(g)
    nb_para = np.array(para.a)
    
    # On prépare le tableau   
    
    df_edges = pd.DataFrame(g.get_edges(),columns=['from','to','index']) # tableau complet des arêtes
    d_e = pd.DataFrame(np.arange(len(nb_para)),columns=['e_index']) # tableau des arêtes outliers  
    
    d_e.set_index('e_index',inplace=True) # changement des indexs pour un inner join
    df_edges.set_index('index',inplace=True) # changement des indexs pour un inner join
    
    df_edges_out = d_e.merge(df_edges,left_index=True,right_index=True) # jointure interieure
        
    # On réencode les adresses
    df_edges_out['from_address'] = Encoder.inverse_transform(df_edges_out['from'])
    df_edges_out['to_address'] = Encoder.inverse_transform(df_edges_out['to'])
    df_edges_out['number_parallel_transactions'] = nb_para
    
    return(df_edges_out)

In [186]:
par_df = parallel_tr(g)

In [158]:
par_df.head()

,from,to,from_address,to_address,number_parallel_transactions
0,804365,1557070,0x7ed1e469fcb3ee19c0366d829e291451be638e59,0xf72d552ad28e2ea7db6ca5794d359aec190d9151,0
1,326571,197526,0x32be343b94f860124dc4fee278fdcbd38c102d88,0x1e49a5551a183d16451982729ba03e5887364e58,0
2,327749,1471263,0x32ec262a2022ba5adca1e3ad1b1fe85be63fdcdd,0xe94b04a0fed112f3664e45adb2b8915693dd5ff3,0
3,1519051,1471263,0xf10a0907bc4d0953915c27641c8583aea8d78537,0xe94b04a0fed112f3664e45adb2b8915693dd5ff3,0
4,447488,1471263,0x4614ecb1e4afd8fae805eb3e5548579e4d039464,0xe94b04a0fed112f3664e45adb2b8915693dd5ff3,0


In [159]:
data.head()

,hash,nonce,block_hash,block_number,transaction_index,from_address,to_address,value,gas,gas_price,input
0,0x0bfb813c50d7e37b7c37df5ee33f90a659e0687071e0...,81417,0x9d15648d01925f689f66d689299b33ac6a3a86286782...,4100200,0,804365,1557070,1.620000e+17,21000,60000000000,0x
1,0x165a95bb79d39597ca95eaae45d4baae1828deb1d82c...,630827,0x9d15648d01925f689f66d689299b33ac6a3a86286782...,4100200,1,326571,197526,4.924194e+17,100000,51000000000,0x
2,0x18e5ac13b126f43c25c5c443f5a967336a3e58b077b0...,2,0x9d15648d01925f689f66d689299b33ac6a3a86286782...,4100200,2,327749,1471263,4.982420e+17,135546,51000000000,0x0f2c9329000000000000000000000000fbb1b73c4f0b...
3,0x79c9060ad01926cab66835a615cfa0326f4bcaa505f8...,0,0x9d15648d01925f689f66d689299b33ac6a3a86286782...,4100200,3,1519051,1471263,6.778990e+17,135546,51000000000,0x0f2c9329000000000000000000000000fbb1b73c4f0b...
4,0x828ed0499dfa8793d7d97a0c113667ed86fef258115e...,2,0x9d15648d01925f689f66d689299b33ac6a3a86286782...,4100200,4,447488,1471263,1.815742e+18,135546,51000000000,0x0f2c9329000000000000000000000000fbb1b73c4f0b...


In [193]:
par_df

,from,to,from_address,to_address,number_parallel_transactions
0,804365,1557070,0x7ed1e469fcb3ee19c0366d829e291451be638e59,0xf72d552ad28e2ea7db6ca5794d359aec190d9151,0
1,326571,197526,0x32be343b94f860124dc4fee278fdcbd38c102d88,0x1e49a5551a183d16451982729ba03e5887364e58,0
2,327749,1471263,0x32ec262a2022ba5adca1e3ad1b1fe85be63fdcdd,0xe94b04a0fed112f3664e45adb2b8915693dd5ff3,0
3,1519051,1471263,0xf10a0907bc4d0953915c27641c8583aea8d78537,0xe94b04a0fed112f3664e45adb2b8915693dd5ff3,0
4,447488,1471263,0x4614ecb1e4afd8fae805eb3e5548579e4d039464,0xe94b04a0fed112f3664e45adb2b8915693dd5ff3,0
5,561080,1471263,0x5837cb5d40a0ad9a92c61f670a30de7a05ac93a7,0xe94b04a0fed112f3664e45adb2b8915693dd5ff3,0
6,1024326,1471263,0xa1e8d3d616516a7a9b64e75ffae7e3ab36f943ef,0xe94b04a0fed112f3664e45adb2b8915693dd5ff3,0
7,829970,1471263,0x82ebc9ba24b9ea75a334367b49c54db6f96fa2ef,0xe94b04a0fed112f3664e45adb2b8915693dd5ff3,0
8,1178808,1471263,0xba7c43a82738ce71164233db9619c10683fff6f4,0xe94b04a0fed112f3664e45adb2b8915693dd5ff3,0
9,326571,1272664,0x32be343b94f860124dc4fee278fdcbd38c102d88,0xc96d400ecc44dfec99742e91d99f54d2cbc47651,0


In [194]:
par_df2 = par_df.copy()


In [195]:
shortest_distance(g,326571,1272664)

1

In [205]:
exc_df = parse_all_exchanges()

In [206]:
exc_df

,addresses,name,balance,txn
0,0x05f51aab068caa6ab7eeb672f88c180f67f17ec7,ABCC,7.084121e+02,47514
1,0x7a10ec7d68a048bdae36a70e93532d31423170fa,Bgogo 1,1.873394e+02,17990
2,0xce1bf8e51f8b39e51c6184e059786d1c0eaf360f,Bgogo 2,1.400177e+03,7723
3,0xf73c3c65bde10bf26c2e1763104e609a41702efe,Bibox,8.991313e+02,535269
4,0xa30d8157911ef23c46c0eb71889efe6a648a41f7,BigONE,7.612947e+02,117626
5,0xf7793d27a1b76cdf14db7c83e82c772cf7c92910,Bilaxy,3.613385e+02,150371
6,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,Binance 1,3.732828e+04,8894643
7,0xd551234ae421e3bcba99a0da6d736074f22192ff,Binance 2,5.056743e+03,1460005
8,0x564286362092d8e7936f0549571a803b203aaced,Binance 3,6.536515e+03,1369382
9,0x0681d8db095565fe8a346fa0277bffde9c0edbbf,Binance 4,7.331578e+03,1433506
